## Web Application with Django 2

### Building A Blog App (CRUD)
In this lesson, we will build a Blog application that allows users to **Create, Read, Update and Delete** posts.

The homepage will read or list all blog posts  and there will be a dedicated detail page for each individual
blog post. The detail page with also enable functionality such as Edit and Delete of post.

We’ll also introduce CSS for styling and install a predesigned theme.

##### Initial Set Up
The steps for setting up a new Django project are as follows:
- Create a new virtual environment with `conda` called `blogapp` and activate it
- Make a new directory for our code called `blog` inside our virtual environment
- Install Django in the new virtual environment.
- create a new Django project called `blog_project`
- create a new app blog
- perform a migration to set up the database
- update blog_project/settings.py

Let’s implement them now in a new command line terminal.

In [ ]:
# blog_project/settings.py

INSTALLED_APPS = [
"django.contrib.admin",
"django.contrib.auth",
"django.contrib.contenttypes",
"django.contrib.sessions",
"django.contrib.messages",
"django.contrib.staticfiles",
"blog.apps.BlogConfig", # new
]


In [ ]:
 python manage.py runserver

##### Database Models
What are the characteristics of a typical blog application? In our case, let’s keep things simple
and assume each post has a 
- title,
- author,
- body,
- publish date and 
- create_date. 

We can turn this into a database model by opening the `blog/models.py` file and entering the code below:

In [ ]:
# blog/models.py

from django.db import models
from django.urls import reverse
from django.utils import timezone
from django.contrib.auth.models import User


class Post(models.Model):
    
    title = models.CharField(max_length=200)
    author = models.ForeignKey(User,on_delete=models.CASCADE, related_name='blog_posts' )
    body = models.TextField()
    publish = models.DateTimeField(default=timezone.now)
    created = models.DateTimeField(auto_now_add=True)

    class Meta:
      # Specifies the default ordering of posts in queries as per the 'publish' field.
      ordering = ['-publish']
      
      
    def __str__(self):
      # Returns a string representation of the post object, using its title.
      return self.title
  
    def get_absolute_url(self):
       # Returns the canonical URL for a post detail view,
       # using the 'post_detail' URL pattern and passing the post's primary key as a parameter.
     return reverse("post_detail", kwargs={"pk": self.pk})

##### Creating and Applying Migrations

Now that we have a data model for blog posts, we need to create the corresponding database table. 

Django comes with a migration system that tracks the changes made to models and enables them to 
propagate into the database.

The migrate command applies migrations for all applications listed in **INSTALLED_APPS**. It synchronizes the database with the current models and existing migrations.

First, we will need to create an initial migration for our `Post model`. 

Run the following command in the shell prompt from the root directory of your project. And remember we need to do this each time we crete a model or make an update to our model.

In [ ]:
# Shell Command

python manage.py makemigrations blog
python manage.py migrate

##### Creating an administration site for models
Now that the Post model is in sync with the database, we can create a simple administration site to 
manage blog posts.

Django comes with a built-in administration interface that is very useful for editing content. The Django 
site is built dynamically by reading the model metadata and providing a production-ready interface for 
editing content. You can use it out of the box, configuring how you want your models to be displayed in it.

The ***django.contrib.admin*** application is already included in the **INSTALLED_APPS** setting, so you 
don’t need to add it.

##### Create a Superuser

In [ ]:
python manage.py createsuperuser
Username (leave blank to use 'wsv'): james
Email:jamezlim90@gmail.com
Password:
Password (again):
Superuser created successfully.

Run `python manage.py runserver` to view the admin interface. At the moment the blog post model we defined is not registered or activated in the admin dashboard yet.

##### Adding models to the administration site
Let’s add your blog models to the administration site. Edit the admin.py file of the blog application 
and make it look like this.

In [ ]:
# blog/admin.py

from django.contrib import admin
from .models import Post


admin.site.register(Post)

##### Adding post object in the admin 
Let’s add 3 blog posts so we have some sample data to work with.

Click on the + Add button next to Posts to create a new entry. Make sure to add an “author” to each post too since by default all model fields are required without it you will get an error.

#### URLs, Views and Templates

Now that our database model is complete we need to create the necessary views, URLs, and
templates so we can display the information on our web application.

In your text editor, create a new file called `urls.py` within the blog app and update it with the
code below

In [ ]:
# blog/urls.py

from django.urls import path
from .views import BlogListView

urlpatterns = [
path("", BlogListView.as_view(), name="home"),
]


We also should update our django_project/urls.py file so that it knows to forward all requests
directly to the blog app.

In [ ]:
# django_project/urls.py

from django.contrib import admin
from django.urls import path, include # new

urlpatterns = [
    
path("admin/", admin.site.urls),
path("", include("blog.urls")), # new

]

We’ve added include on the second line and a URL pattern using an empty string regular
expression, "", indicating that URL requests should be redirected as is to blog’s URLs for further
instructions.


##### Views
We’re going to use class-based views but if you want to see a function-based way to build a blog
application

In our views file, add the code below to display the contents of our Post model using ListView.

In [ ]:
# blog/views.py

from django.views.generic import ListView
from .models import Post

class BlogListView(ListView):
    model = Post
    template_name = "blog/home.html"

On the top two lines we import ListView and our database model Post.

Then we subclass `ListView` and add links to our model and template. This saves us a lot of code versus implementing it all from scratch.

##### Templates
With our `URLs and views` now complete, we’re only missing the third piece of the puzzle:
templates.

We will start by creating our new templates directory. And then create our `base.html` file as the parent template to be inherited by other child templates.

After that we will create a blog directory inside the templates directory to hold all our `blog app` templates

In [ ]:
# Shell Command

mkdir templates
cd templates
code base.html
mkdir blog
cd blog
code home.html

Then update blog_project/settings.py so Django knows to look there for our templates.

In [ ]:
# blog_project/settings.py
TEMPLATES = [
{
...
"DIRS": [BASE_DIR / "templates"], # new
...
},
]


Update the base.html template as follows:


In [ ]:
<!-- templates/base.html -->
{% load static %}

<html>
    <head>
        <title>Django blog</title>
        <link href="https://fonts.googleapis.com/css?family=Source+Sans+Pro:400" rel="stylesheet">
        <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.3/dist/css/bootstrap.min.css">
        <link href="{% static 'css/base.css' %}" rel="stylesheet">
    </head>
    <body>
        <header>
            <nav class="navbar navbar-expand-lg bg-body-tertiary bg-primary" data-bs-theme="dark">
                <div class="container-fluid">
                  <a class="navbar-brand" href="{% url 'home' %}">Django blog</a>
                  <button
                    class="navbar-toggler"
                    type="button"
                    data-bs-toggle="collapse"
                    data-bs-target="#navbarSupportedContent"
                    aria-controls="navbarSupportedContent"
                    aria-expanded="false"
                    aria-label="Toggle navigation"
                  >
                    <span class="navbar-toggler-icon"></span>
                  </button>
                  <div class="collapse navbar-collapse" id="navbarSupportedContent">
                    <ul class="navbar-nav me-auto mb-2 mb-lg-0">
                      <li class="nav-item">
                        <a class="nav-link active" aria-current="page" href="{% url 'home' %}">Home</a>
                      </li>
                    </ul>

                    {% comment %}
                    <div class="d-flex">
                      <a href="{% url 'post_new' %}">+ New Blog Post</a>
                    </div>
                    {% endcomment %} 
                  </div>
                </div>
              </nav>
        <h1></h1>
        </header>
        <div>
        {% block content %}
        {% endblock content %}
        </div>

      <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.3.3/dist/js/bootstrap.min.js"></script>   
    </body>
</html>

Note that code between {% block content %} and {% endblock content %} can be filled by
other templates. Speaking of which, here is the code for home.html.


In [ ]:
<!-- templates/home.html -->

{% extends "base.html" %}

{% block content %}
<div class= "container mx-auto w-70 bg-primary m-3 p-3 text-center text-light">
  <h2>Blog Posts</h2>
</div>
<div class="list-group container mx-auto w-70 mt-3">
    
{% for post in post_list %}
<a href="#" class="list-group-item list-group-item-action" aria-current="true">
    <div class="d-flex w-100 justify-content-between">
      <h5 class="mb-1">{{ post.title }}</h5>
      <small>{{post.publish}}</small>
    </div>
    <small class="mb-2">Author : {{post.author}}</small>
    <p class="mb-1">{{ post.body }}</p>
  
  </a>
{% endfor %}
</div>

{% endblock content %}
    

##### Static Files

Now we need to add static files to our project. To do that we will take the following steps:

- Make directory called `static`
- create the relevant **CSS, Images and Javascript** files inside the static directory
- Update the settings.py file in the blog_project directory with the following code
- lastly, we will update our `base.html` template to link to our `styles.css` file

In [ ]:
# Shell command

mkdir static
cd Static
mkdir css && cd css
code base.css

In [ ]:
# django_project/settings.py
STATIC_URL = "/static/"
STATICFILES_DIRS = [BASE_DIR / "static"] # new

- Lastly, We need to add the static files to our templates by adding `{% load static %}` to
the top of base.html. Because our other templates inherit from base.html.
-  We only have to add this once. Include a new line at the bottom of the <head></head> code that explicitly references our new base.css file.

#### Detail Blog Pages
Now we can add the functionality for individual blog pages called detail view.

To do that? We need to create a new `view, url, and template`. to handle the functionality.

In [ ]:
# blog/views.py

from django.views.generic import ListView, DetailView # new

...
class BlogDetailView(DetailView): # new
    model = Post
    template_name = "blog/post_detail.html"

In [ ]:
<!-- templates/blog/post_detail.html -->

{% extends "base.html" %}

{% block content %}

<div>
     <!-- About section-->
     <section id="">
        <div class="container px-4">
            <div class="row gx-4 justify-content-center">
                <div class="col-lg-8">
                    <h2>{{ post.title }}</h2>
                    <small>{{post.author}}</small>
                    <br>
                    <small>{{post.publish}}</small>
                    <hr>
                    <p class="lead">{{ post.body }}</p>
                    
                </div>
            </div>
        </div>
    </section>
</div>

{% endblock content %}

In [ ]:
# blog/urls.py


from .views import BlogListView, BlogDetailView # new

urlpatterns = [
    
path("post/<int:pk>/", BlogDetailView.as_view(), name="post_detail"), # new
path("", BlogListView.as_view(), name="home"),

]


To make our life easier, we should update the link on the homepage so we can directly access
individual blog posts from there.

Swap out the current empty link, `<a href="">, for <a href="{%url 'post_detail' post.pk %}">`

In [ ]:
<a href="{% url 'post_detail' post.pk %}" class="list-group-item list-group-item-action" aria-current="true">
    
</a>

#### Testing
To begin we can set up our test data and check the Post model’s content. Here’s how that might
look.


In [ ]:
# blog/tests.py
from django.contrib.auth import get_user_model
from django.test import TestCase
from django.urls import reverse # new
from .models import Post


class BlogTests(TestCase):
    
    @classmethod
    def setUpTestData(cls):
        cls.user = get_user_model().objects.create_user(username="testuser", email="test@gmail.com", password="secret")
        cls.post = Post.objects.create(title="Blog", body="This is a blog", author=cls.user,)
        
    def test_post_model(self):
        self.assertEqual(self.post.title, "Blog")
        self.assertEqual(self.post.body, "This is a blog")
        self.assertEqual(self.post.author.username, "testuser")
        self.assertEqual(str(self.post), "Blog")
        self.assertEqual(self.post.get_absolute_url(), "/post/1/")
        
        
    def test_url_exists_at_correct_location_listview(self): # new
        response = self.client.get("/")
        self.assertEqual(response.status_code, 200)
        
    def test_url_exists_at_correct_location_detailview(self): # new
        response = self.client.get("/post/1/")
        self.assertEqual(response.status_code, 200)
        
    def test_post_listview(self): # new
        response = self.client.get(reverse("home"))
        self.assertEqual(response.status_code, 200)
        self.assertContains(response, "This is a blog")
        self.assertTemplateUsed(response, "blog/home.html")
        
        
    def test_post_detailview(self): # new
        response = self.client.get(reverse("post_detail", kwargs={"pk": self.post.pk}))
        no_response = self.client.get("/post/100000/")
        self.assertEqual(response.status_code, 200)
        self.assertEqual(no_response.status_code, 404)
        self.assertContains(response, "This is a blog")
        self.assertTemplateUsed(response, "blog/post_detail.html")

#### Initialize with Git
Now is also a good time for our first Git commit. First, initialize our directory. Then make a commit and push to origin with the following code


In [ ]:
git init
git status
git add -A
git commit -m "initial commit"


### Django Forms

We will continue to develop our django blog application. We have learnt how to implement a blog listings, detail pages, testing and pushing to Github.

Next, we’ll add forms for creating and editing models in the frontend. so we don’t have to use the Django admin at all for these changes.

Django comes with powerful built-in Forms that abstract away much of the difficulty for us. Django also comes with generic editing views for common tasks like displaying, creating, updating, or deleting a form.

##### Steps to Creating a Form in Django (CreateView)
Creating a form in django is very simple it follows the same urls, views and template approach we have learnt so far. The steps are listed below:

- Firstly, we create a add a new url to the list of `url_pattern` variable in `blog/urls.py` file to handle the form creation request from a user with the given name to be used in the template.
- We create our view by importing and inheriting from a new generic class called **CreateView** to help handle the form creation process.
- The last step is to create our template called `post_new.html` in the text editor and also update the `base.html` file to capture the form link.

So lets dive into code...,

In [ ]:
# blog/urls.py

from django.urls import path
from .views import BlogListView, BlogDetailView, BlogCreateView # new

urlpatterns = [
    
path("post/new/", BlogCreateView.as_view(), name="post_new"), # new
path("post/<int:pk>/", BlogDetailView.as_view(), name="post_detail"),
path("", BlogListView.as_view(), name="home"),

]


In [ ]:
from django.views.generic.edit import CreateView # new

class BlogCreateView(CreateView): # new
    model = Post
    template_name = "blog/post_new.html"
    fields = ["title", "author", "body"]

In [ ]:
<!-- templates/blog/post_new.html -->

{% extends "base.html" %}

{% block content %}

<div class="container mx-auto w-60">
    <h3>New post</h3>
    <form action="" method="post">
    {% csrf_token %}
    {{ form.as_p }}
    <input type="submit" value="Save" class="btn-success btn">
    </form>
    </div>

{% endblock content %}

Lastly, update the base.html by uncommenting the link to create a new post.

##### Django-Crispy-Form

Inorder to maake our form look prettier and easy to style we'll rely on a two third party packages.`django-crispy-forms` and ` Bootstrap5 template pack`

In the Terminal run the following command for the installations

In [ ]:
python -m pip install django-crispy-forms
python -m pip install crispy-bootstrap5

Next we'll add the packages to our INSTALLED_APPS in the settings.py file

In [ ]:
# blog_project/settings.py


INSTALLED_APPS = [
"django.contrib.admin",
"django.contrib.auth",
"django.contrib.contenttypes",
"django.contrib.sessions",
"django.contrib.messages",
"django.contrib.staticfiles",

# 3rd Party
"crispy_forms", # new
"crispy_bootstrap5", # new

# Local
"blog"

]

And then at the bottom of the settings.py file add two new lines as well.

In [ ]:
# blog_project/settings.py

CRISPY_ALLOWED_TEMPLATE_PACKS = "bootstrap5" # new
CRISPY_TEMPLATE_PACK = "bootstrap5" # new

Now in our` post_new.html` template we can quickly use crispy forms to make things look prettier.

First, we load `crispy_forms_tags` at the top and then swap out `{{ form.as_p }}` for `{{ form|crispy }}`. 

We’ll also update the **Save** button to be green with the `btn-primary` styling.

In [ ]:
<!-- templates/blog/post_new.html -->

{% extends "base.html" %}
{% load crispy_forms_tags %}    


{% block content %}

<h3>New post</h3>
<form action="" method="post">
{% csrf_token %}
{{ form|crispy }}

<input type="submit" value="Save" class="form-control btn btn-primary">
</form>

{% endblock content %}

##### Steps to Updating a Form in Django (UpdateView)
The process for creating an update form so users can edit blog posts is similar to the create view. 

We’ll again use a built-in Django class-based generic view, `UpdateView`, and create the requisite
***template, url, and view.***

To start, let’s add a new link to `post_detail.html` so that the option to edit a blog post appears
on an individual blog page.


So lets dive into code...,

In [ ]:
<!-- templates/blog/post_detail.html -->

{% extends "base.html" %}
{% block content %}

<div class="post-entry">
<h2>{{ post.title }}</h2>
<p>{{ post.body }}</p>
</div>

<a href="{% url 'post_edit' post.pk %}">+ Edit Blog Post</a>
{% endblock content %}

Next we create the template file for our edit page called `post_edit.html` and add the following
code

In [ ]:
<!-- templates/blog/post_edit.html -->

{% extends "base.html" %}
{% block content %}

<div class="container mx-auto w-50 "></div>
<h3>Edit post</h3>
<form action="" method="post">
{% csrf_token %}
{{form|crispy}}
<input type="submit" value="Update" class="btn btn-primary">
</form>
</div>

{% endblock content %}


Now to our view. We need to import UpdateView on the second-from-the-top line and then
subclass it in our new view `BlogUpdateView`

In [ ]:
# blog/views.py

from django.views.generic.edit import CreateView, UpdateView # new

class BlogUpdateView(UpdateView):           # new
    model = Post
    template_name = "blog/post_edit.html"
    fields = ["title", "body"]


The final step is to update our `urls.py` file as follows. Add the `BlogUpdateView` up top and then
the new route at the top of the existing urlpatterns as follows

In [ ]:
from .views import BlogListView,BlogDetailView,BlogCreateView, BlogUpdateView, #new


urlpatterns = [
...
path("post/<int:pk>/edit/", BlogUpdateView.as_view(), name="post_edit"), # new

]

##### DeleteView

The process for creating a form to delete blog posts is very similar to that for updating a post.

We’ll use yet another generic class-based view, `DeleteView`, And then create the necessary view, url, and
template.

Let’s start by adding a link to delete blog posts on our individual blog page,` post_detail.html`.

In [ ]:
<!-- templates/blog/post_detail.html -->

{% extends "base.html" %}
{% block content %}

<div class="">
<h2>{{ post.title }}</h2>
<p>{{ post.body }}</p>
</div>
<p><a href="{% url 'post_edit' post.pk %}">+ Edit Blog Post</a></p>
<p><a href="{% url 'post_delete' post.pk %}">+ Delete Blog Post</a></p>

{% endblock content %}


Then create a new file for our delete page template. It will be called templates/blog/post_delete.html
and contain the following code:

In [ ]:
<!-- templates/blog/post_delete.html -->

{% extends "base.html" %}
{% block content %}

<div class="container mx-auto w-50 ">
    <h3>Delete post</h3>
    <form action="" method="post">
    {% csrf_token %}
    <p class="lead">Are you sure you want to delete "{{ post.title }}"?</p>
    
    <input type="submit" value="Confirm" class="btn btn-danger">
    </form>
</div>

{% endblock content %}

Now update the `blog/views.py` file, by importing `DeleteView` and `reverse_lazy` at the top, then
create a new view that subclasses `DeleteView`.


In [ ]:
# blog/views.py

from django.views.generic.edit import CreateView, UpdateView, DeleteView # new
from django.urls import reverse_lazy # new


...
class BlogDeleteView(DeleteView): # new
    model = Post
    template_name = "blog/post_delete.html"
    success_url = reverse_lazy("home")

As a final step, create a URL by importing our view `BlogDeleteView` and adding a new pattern:


In [ ]:
from .views import (
BlogListView,
BlogDetailView,
BlogCreateView,
BlogUpdateView,
BlogDeleteView, # new
)

urlpatterns = [
    ...
    path("post/<int:pk>/delete/", BlogDeleteView.as_view(), name="post_delete"), # new
   
]

#### Pushing to GitHub

With a small amount of code we’ve built a Blog application that allows for creating, reading, updating, and deleting blog posts. 

This core functionality is known by the acronym CRUD: ***Create-Read-Update-Delete***. While there are multiple ways to achieve this same functionality–we could have used function-based views or written our own class-based views–we’ve demonstrated how
little code it takes in Django to make this happen.

Next is we push our code to GitHub using the following Commands and the desktop interface.

In [ ]:

git status
git add .
git commit -m "added crud functionality"


### Django Authentication
So far we’ve built a working blog application with forms but we’re missing a major piece of most
web applications: user authentication.

Implementing proper user authentication is famously hard; there are many security gotchas
along the way so you really don’t want to implement this yourself.

Fortunately, Django comes with a powerful, built-in user authentication system that we can use and customize as needed.

Whenever you create a new project, by default Django installs the `auth app`, which provides us
with a User object99 containing:

- username
- password
- email
- first_name
- last_name

We will use this User object to implement log in, log out, and sign up in our blog application

Django provides us with a default view for a log in page via `LoginView`. 

All we need to add a URL pattern for the auth system, a log in template, and a small update to our
`blog_project/settings.py` file.

First, update the `django_project/urls.py` file. We’ll place our log in and log out pages at the
`accounts/ URL`. This is a one-line addition on the next-to-last line

In [ ]:
# django_project/urls.py
from django.contrib import admin
from django.urls import path, include


urlpatterns = [
...
path("accounts/", include("django.contrib.auth.urls")), # new
..
]


Next Let's make our login page! By default, Django will look within a templates folder called `registration` for auth templates. The login template is called `login.html`.

In [ ]:
mkdir templates/registration
cd registration
ni login.html
ni signup.html

Then we include the following code in the `login.html` file

In [ ]:
{% extends "base.html" %}
{% load crispy_forms_tags %}

{% block content %}
<div class="container mx-auto w-50 ">
<h3>Log In</h3>
<form method="post">
  {% csrf_token %}
  {{ form|crispy }}
  <button type="submit" class="btn btn-primary">Log In</button>
</form>
</div>
{% endblock content %}

#### Log Out Button

Now we add a `Logout` button to the `base.html` file  under the `<ul>` section. We will update the file using the following code 

In [ ]:
{% if user.is_authenticated %}
                    
<div class="d-flex text-light mx-3">
    Hi {{ user.username }}!
</div>
<div class="d-flex text-light mx-3">
    <a href="{% url 'post_new' %}">+ New Blog Post</a>
</div> 
</div>
<form action="{% url 'logout' %}" method="post" class="d-flex">
{% csrf_token %}
<button class="btn btn-outline-danger mt-3" type="submit">Log Out</button>
</form>
{% else %}
<a href="{% url 'login' %}" class="btn btn-primary mt-3">Log In</a>
{% endif %}

Then update `settings.py` with our redirect link, `LOGIN_REDIRECT_URL` and `LOGOUT_REDIRECT_URL`. Add it right next to our login redirect so the bottom of the settings.py file should look as follows:

In [ ]:
LOGIN_REDIRECT_URL = "home"  # new
LOGOUT_REDIRECT_URL = "home"  

#### Sign Up Page

Now that we have sorted out logging in and logging out, it is time to add a signup page to our basic Django site. If you recall, Django does not provide a built-in view or URL for this, so we must code up the form and the page ourselves.

To begin, stop the local webserver with Control+c and create a dedicated app called `accounts`, which we'll use for our custom account logic.

In [ ]:
python manage.py startapp accounts

Make sure to add the new app to the `INSTALLED_APPS` setting in the blog_project/settings.py file:

In [ ]:
# django_project/settings.py

INSTALLED_APPS = [
    "django.contrib.admin",
    "django.contrib.auth",
    "django.contrib.contenttypes",
    "django.contrib.sessions",
    "django.contrib.messages",
    "django.contrib.staticfiles",
    "accounts",  # new
]

Then add a URL path in `blog_project/urls.py` that is above our included Django auth app. The order is important here because Django looks for URL patterns from top-to-bottom.

In [ ]:
# blog_project/urls.py
from django.contrib import admin
from django.urls import path, include

urlpatterns = [
    path("admin/", admin.site.urls),
    path("accounts/", include("accounts.urls")),  # new
    path("accounts/", include("django.contrib.auth.urls")),
    
]

Next, we create a new file called `accounts/urls.py` with your text editor and add the following code.

In [ ]:
# accounts/urls.py
from django.urls import path

from .views import SignUpView


urlpatterns = [
    path("signup/", SignUpView.as_view(), name="signup"),
]

Now for the accounts/views.py file:

In [ ]:
# accounts/views.py
from django.contrib.auth.forms import UserCreationForm
from django.urls import reverse_lazy
from django.views.generic import CreateView


class SignUpView(CreateView):
    form_class = UserCreationForm
    success_url = reverse_lazy("login")
    template_name = "registration/signup.html"

Ok, now for the final step. Create a new template, `templates/registration/signup.html`, and populate it with this code that looks almost exactly like what we used for login.html.

In [ ]:
<!-- templates/registration/signup.html -->
{% extends "base.html" %}
{% load crispy_forms_tags %}


{% block content %}
<div class="container mx-auto w-50 ">
<h3>Sign Up</h3>
<form method="post">
  {% csrf_token %}
  {{ form|crispy }}
  <button type="submit" class="btn btn-primary">Sign Up</button>
</form>
</div>
{% endblock %}



We're done! To confirm it all works, spin up our local server with python manage.py runserver and navigate to http://127.0.0.1:8000/accounts/signup/.

##### SignUp link
Add the link for ***Sign Up*** just below the existing link for “Log In” as follows:

In [ ]:
...
{% else %}
<a href="{% url 'login' %}" class="btn btn-primary mt-3">Log In</a>
<a href="{% url 'signup' %}">Sign Up</a>
{% endif %}
